In [1]:
import torch
import torchvision
import torchvision.transforms.functional as TF

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Dec 20 09:45:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2): 
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2] 

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip) 

        return self.final_conv(x)

In [5]:
# google drive를 mount 시키기 (데이터셋 연동을 위함)
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [6]:
import os
os.chdir('/content/drive/My Drive/TrainingData/TrainingData')

In [7]:
!pip install albumentations==0.4.6

     |████████████████████████████████| 117 kB 9.8 MB/s 
     |████████████████████████████████| 948 kB 50.4 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65172 sha256=66745266b840d7f7ba1ca8bcbd41b330e88ddc8f33a3b1bea53074a893921b25
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [11]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from utils import (
    load_checkpoint,
    save_checkpoint,
    get_loaders,
    check_accuracy,
    save_predictions_as_imgs,
)
import gc
gc.collect()
torch.cuda.empty_cache()

# Hyperparameters etc.
LEARNING_RATE = 1e-3  #과대적합 되는 것 확인했으므로 lr은 2이하로 낮출 것
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 4
NUM_EPOCHS = 6
NUM_WORKERS = 2
IMAGE_HEIGHT = 1200  # 1280 originally
IMAGE_WIDTH = 600  # 1918 originally
PIN_MEMORY = True
LOAD_MODEL = True
TRAIN_IMG_DIR = "png/img/" #train data
TRAIN_MASK_DIR = "train/npy_label/" #train label
VAL_IMG_DIR = "testpng/" #test data
VAL_MASK_DIR = "train/npy_label/"

def train_fn(loader, model, optimizer, loss_fn, scaler, i): #train function
    loop = tqdm(loader) #loop에 설명문 추가

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # Forward
        with torch.cuda.amp.autocast():
            predictions = model(data) #model에 data를 넣은게 예측치이다.
            loss = loss_fn(predictions, targets[...,i]) #예측이랑 타겟이 다른 정도가 loss이다. test할 때는 이 loss는 필요 없이 prediction만 만들어서 npy로 저장하면 된다.

        # Backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Update tqdm loop
        loop.set_postfix(loss=loss.item())

def main(i):
    train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.OneOf([
                     A.MotionBlur(p=1),
                     A.OpticalDistortion(p=1),
                     A.GaussNoise(p=1),
            ]),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )



    # Validation 시 적용하는 augmentation.
    val_transforms = A.Compose(
        [   
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,),
            ToTensorV2(),
        ],
    )

    ## Model 및 loss function, optimizer 정의.
    model = UNET(in_channels=3, out_channels=1).to(DEVICE)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    ## Data loader 정의.-> 이 부분 수정해야함
    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

    if LOAD_MODEL:
        load_checkpoint(torch.load(f"my_checkpoint{i}_pretrained.pth.tar"), model)

    check_accuracy(val_loader, model, i, device=DEVICE) #이 것을 통해 dice coefficient를 출력하는데 약간의 오류가 있어 일단 오류 없이 결과제출을 위해 주석처리 해두었습니다. 
    scaler = torch.cuda.amp.GradScaler()

    
    for epoch in range(NUM_EPOCHS): #train진행
      train_fn(train_loader, model, optimizer, loss_fn, scaler, i)
      print(epoch)
        

        # Save model
      checkpoint = {
          # Pytorch에서 모델의 state_dict은 학습가능한 매개변수 (weight & bias)가 담겨있는 딕셔너리(Dictionary)입니다. 
          "state_dict": model.state_dict(),
          "optimizer":optimizer.state_dict(),
      }
      save_checkpoint(checkpoint, f"my_checkpoint{i}_pretrained.pth.tar")

if __name__ == "__main__":
    main(0)
    main(1)
    main(2)
    main(3)
    main(4)
    main(5)
    main(6)

    

=> Loading checkpoint
Got 42128811/42480000 with acc 99.17
Dice score: 0.0


100%|██████████| 30/30 [01:05<00:00,  2.19s/it, loss=0.0546]


0
=> Saving checkpoint


100%|██████████| 30/30 [01:04<00:00,  2.16s/it, loss=0.0385]


1
=> Saving checkpoint


...

3
=> Saving checkpoint


100%|██████████| 30/30 [01:03<00:00,  2.12s/it, loss=0.599]


4
=> Saving checkpoint


100%|██████████| 30/30 [01:02<00:00,  2.08s/it, loss=0.643]


5
=> Saving checkpoint


In [12]:
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_EPOCHS = 3
NUM_WORKERS = 2
IMAGE_HEIGHT = 1000  # 1280 originally
IMAGE_WIDTH = 500  # 1918 originally
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = "png/img/" #train data
TRAIN_MASK_DIR = "train/npy_label/" #train label
VAL_IMG_DIR = "testpng/" #test data

import gc
gc.collect()
torch.cuda.empty_cache()

model_1 = UNET(in_channels=3, out_channels=1).to(DEVICE)
model_2 = UNET(in_channels=3, out_channels=1).to(DEVICE)
model_3 = UNET(in_channels=3, out_channels=1).to(DEVICE)
model_4 = UNET(in_channels=3, out_channels=1).to(DEVICE)
model_5 = UNET(in_channels=3, out_channels=1).to(DEVICE)
model_6 = UNET(in_channels=3, out_channels=1).to(DEVICE)
model_7 = UNET(in_channels=3, out_channels=1).to(DEVICE)
                                                            #저장되어있는 trained model을 이런 식으로 불러온다.
m1_data=torch.load('my_checkpoint0_pretrained.pth.tar')
m2_data=torch.load('my_checkpoint1_pretrained.pth.tar')
m3_data=torch.load('my_checkpoint2_pretrained.pth.tar')
m4_data=torch.load('my_checkpoint3_pretrained.pth.tar')
m5_data=torch.load('my_checkpoint4_pretrained.pth.tar')
m6_data=torch.load('my_checkpoint5_pretrained.pth.tar')
m7_data=torch.load('my_checkpoint6_pretrained.pth.tar')

model_1.load_state_dict(m1_data['state_dict'])
model_2.load_state_dict(m2_data['state_dict'])
model_3.load_state_dict(m3_data['state_dict'])
model_4.load_state_dict(m4_data['state_dict'])
model_5.load_state_dict(m5_data['state_dict'])
model_6.load_state_dict(m6_data['state_dict'])
model_7.load_state_dict(m7_data['state_dict'])

<All keys matched successfully>

In [13]:
import torch
import torchvision
from dataset import (LspineDataset, TestDataset)
from torch.utils.data import DataLoader
import numpy as np
import cv2

import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.optim as optim
from utils import (
    load_checkpoint,
    save_checkpoint,
    get_loaders,
    check_accuracy,
    save_predictions_as_imgs,
)

import gc
gc.collect()
torch.cuda.empty_cache()

LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 1
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = "png/img/" #train data
TRAIN_MASK_DIR = "train/npy_label/" #train label
VAL_IMG_DIR = "testpng/" #test data

In [15]:
gc.collect()
torch.cuda.empty_cache()

test_transforms = A.Compose(
    [   
        A.Resize(height=2400, width=1200),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255,),
        ToTensorV2(),
    ],
)

test_ds = TestDataset(
    image_dir="testpng/",
    mask_dir="testlabel/", #Test는 마스크가 없지만 기존 Traindataset을 수정해서 Dataset을 급하게 만드느라 이걸 남겨뒀습니다. 코드를 따로 정리하지 않고 일단 임의로 train_mask를 넣어서 돌리고 있었습니다.
    transform=test_transforms,
)

test_loader = DataLoader(
    test_ds,
    batch_size=1,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle = False, #주의!...
)





resultarray=[]
for idx, (x,y, a, w, h) in enumerate(test_loader):
  x = x.to(device=DEVICE)
  #print(a, w, h)
          
  #CV2로 Resize를 진행해야 할 것 같다.
  
  with torch.cuda.amp.autocast(): #이제 이 친구를 batch_size_1로 바꾸고 모델 7개 만들어서 test_set뽑고 하면 되겠군
      p1 = np.array(model_1(x).squeeze().tolist()) #model에 data를 넣은게 예측치이다.
      p2 = np.array(model_2(x).squeeze().tolist())
      p3 = np.array(model_3(x).squeeze().tolist())
      p4 = np.array(model_4(x).squeeze().tolist())
      p5 = np.array(model_5(x).squeeze().tolist())
      p6 = np.array(model_6(x).squeeze().tolist())
      p7 = np.array(model_7(x).squeeze().tolist())

      trans=A.Compose([A.Resize(w, h, interpolation=cv2.INTER_LANCZOS4)]) #Resizing하는 부분
      #cv2.INTER_LANCZOS4
      P1=trans(image=p1)['image']
      P2=trans(image=p2)['image']
      P3=trans(image=p3)['image']
      P4=trans(image=p4)['image']
      P5=trans(image=p5)['image']
      P6=trans(image=p6)['image']
      P7=trans(image=p7)['image']

      np_a=np.array([P1,P2,P3,P4,P5,P6,P7])
      result=np.transpose(np_a,(1,2,0))

      
      
      print(result.shape)
      print(result.sum())
      print(result[6].sum())
      print(result)
      #print(f"{a}".replace("['","").replace(".dcm.png", "").replace("']","")+'.npy',result)
      
      
      
      


(3264, 1692, 7)
-147900410.39038977
-45073.195896127894
[[[-6.86583504 -6.13089782 -5.84710081 ... -7.04007279 -4.94121451
    1.51649421]
  [-6.59170277 -5.51896106 -5.41504262 ... -6.80716044 -4.7849541
    1.97637226]
  [-6.34042561 -4.9548231  -4.84258822 ... -6.55682478 -4.65170338
    2.41348531]
  ...
  [-6.03399891 -4.56093068 -4.47066776 ... -6.08897122 -5.70476826
    4.63255382]
  [-6.41253828 -4.88778566 -4.93789155 ... -6.29025314 -5.75042715
    3.60378066]
  [-6.78430029 -5.2227324  -5.57612265 ... -6.61380186 -5.89176659
    2.60961271]]

 [[-6.55273019 -5.33484703 -5.43107968 ... -6.74573424 -4.85560489
    2.0532453 ]
  [-6.13569311 -5.09182515 -4.85800751 ... -6.43106639 -4.86063218
    2.69590033]
  [-5.75955257 -4.79194085 -4.42835094 ... -5.98040437 -4.87064161
    3.26906613]
  ...
  [-5.39057255 -4.78831737 -4.82252443 ... -5.79200938 -6.06376343
    6.44342632]
  [-5.93462006 -4.81676073 -4.82212112 ... -5.94832269 -5.85592549
    4.99981796]
  [-6.4437508  -4.